<a href="https://colab.research.google.com/github/Faabry/Data_Analysis/blob/main/Animes_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><b>Web Scraping Animes</b></h1>
<a href="http://myanimelist.net/topanime.php">http://myanimelist.net/topanime.php</a>

<h3><b>Funções Para Modularizar o Código</b></h3>

In [ ]:
# Função que coleta todas as informações do anime
def getInfo(soup):
    infos = soup.select("div .spaceit_pad")
    info_list = []
    for info in infos:
        info_list.append(info.text.split())
    return info_list

In [ ]:
# Função que extrai o título do anime
def animeName(soup):
    name = soup.select_one(".title-name").text
    if len(name) > 1:
        return name
    return None

In [ ]:
# Função que retorna o rank
def getRank(soup):
    rank = soup.find_all(["div", "span"], {"class": "numbers ranked"})
    for item in rank:
        return item.text

In [ ]:
# Função que extrai o gênero do anime
def animeGenre(information):
    for info in information:
        if (info[0]) == "Genres:":
            return (info[1])
    return None

In [ ]:
# Função que extrai o status (finalizado | em andamento)
def getStatus(information):
    for info in information:
        if (info[0]) == "Status:":
            return (info[1])
    return None

In [ ]:
# Função que retorna a quantidade de episódios
def getEpisodes(information):
    for info in information:
        if info[0] == "Episodes:":
            if info[1] != "Unknown":
                return info[1]
    return None

In [ ]:
# Função que retorna o tema do anime
def getThemes(information):
    for info in information:
        if info[0] == "Themes:":
            return info[1]
    return None

In [ ]:
# Função que retorna o tempo de duração por episódio
def duration(information):
    for info in information:
        if info[0] == "Duration:":
            return info[1]
    return None

In [ ]:
# Função que retorna quantas vezes aquele anime foi favoritado
def getFavorites(information):
    for info in information:
        if info[0] == "Favorites:":
            return info[1].replace(",", ".")
    return None

In [ ]:
# Função que retorna o inicio da transmissão do anime
def startDate(information):
    for info in information:
        if info[0] == "Aired:":
            date = "".join(info[1:4])
            return date.split(",")
    return None

In [ ]:
# Função que retorna o término da transmissão do anime (caso ele tenha sido finalizado)
def endDate(information):
    for info in information:
        if info[0] == "Aired:":
            date = "".join(info[5::])
            return date.split(",")
    return None

<h3><b>Importando e Carregandos os Pacotes Necessários</b></h3>

In [ ]:
# Versão Python
from platform import python_version
print(f"Versão Python Para Este Notebook: {python_version()}")

Versão Python Para Este Notebook: 3.10.9


In [ ]:
# Imports
import bs4
from time import sleep
import json
import requests
from bs4 import BeautifulSoup

In [ ]:
%reload_ext watermark
%watermark -a "Airton Fabre" --iversions

Author: Airton Fabre

requests: 2.28.1
json    : 2.0.9
bs4     : 4.11.1



<h3><b>Extrair os links de cada anime por página</b></h3>

In [ ]:
# Criando loop for para percorrer o número de páginas desejadas
for limit in range(0, 451, 50):
    
    # lista
    links = []

    # Criando uma variável URl
    url = f"https://myanimelist.net/topanime.php?limit={limit}"

    # Extraindo o conteúdo da página
    response = requests.get(url)

    # Formatando o parser da URL
    soup = BeautifulSoup(response.text, "html.parser")

    # Armazenando todas as tags <a> que contém a class "horverinfo_trigger"
    list_of_a = (soup.select(".hoverinfo_trigger a"))

    # Extraindo as informações do attr "href"
    for a in list_of_a:
        links.append(a.get("href"))
    
    # Criando os arquivos.txt com cada link 
    #OBS: cada página terá 50 links
    file = open(f"dados/links/animeslinks{limit}.txt", "w", encoding="utf-8")
    
    # Varrendo os links da lista e armazenando no arquivo.txt
    for link in links:
        file.writelines(link + "\n")
    
    # Fechando o arquivo.txt
    file.close()
    
    # sleep
    sleep(5)

<p>Esse looping percorerrá todos os links dos arquivos.txt que foram gerados no código acima e atráves dos módulos que foram definidos no inicio do script, vamos extrair todas as informações desejadas como nome, gênero etc.
    <br>
PS: Cada página contém 50 animes.<p>


In [ ]:
%%time
# Loop para percorer todas as páginas desejadas
for limit in range(0, 451, 50):
    
    # Dicionário
    animeInfo = []
    # Abrindo o arquivo.txt
    with open(f"dados/links/animeslinks{limit}.txt", "r") as file:
        links = file.readlines()
        
    print("Extraindo os dados, por favor aguarde...")
    
    # Acessando cada link do arquivo.txt
    for link in links:
        # Acessando o link
        resp = requests.get(link)
        
        # Parser do HTML
        soup_info = BeautifulSoup(resp.text, "html.parser")
        
        # Informação dos animes através do módulo "getInfo()"
        information = getInfo(soup_info)
        
        # Adicionando cada dado à sua chave correspondente da lista 
        # e convertendo em dicionário
        animeInfo.append(dict(
            name          = animeName(soup_info),
            ranked        = getRank(soup_info),
            genre         = animeGenre(information),
            theme         = getThemes(information),
            status        = getStatus(information),
            episodes      = getEpisodes(information),
            duration      = duration(information),
            start_date    = startDate(information),
            end_date      = endDate(information),
            favorites     = getFavorites(information)))   
        # Sleep
        sleep(10)

    # Gravando o resultado em disco
    with open(f"dados/AnimeInfo/animeInfo{limit}.txt", 'w') as fout:
        json.dump(animeInfo, fout)
        
    # Sleep
    sleep(10)

Extraindo os dados, por favor aguarde...
Extraindo os dados, por favor aguarde...
Extraindo os dados, por favor aguarde...
Extraindo os dados, por favor aguarde...
Extraindo os dados, por favor aguarde...
Extraindo os dados, por favor aguarde...
Extraindo os dados, por favor aguarde...
Extraindo os dados, por favor aguarde...
Extraindo os dados, por favor aguarde...
Extraindo os dados, por favor aguarde...
CPU times: total: 4min 10s
Wall time: 1h 38min 4s


In [ ]:
print("Web Scraping Concluído com Sucesso !!")

WebScraping Concluído com Sucesso !!


<h1><b>Fim</b></h1>